<a href="https://colab.research.google.com/github/Emanuel-Piovano/hello-world/blob/master/Copia_de_ejemplo_deteccion_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico de detección de objetos
El objetivo de este práctico es entrenar un detector de objetos en imágenes usando la red Yolo.

## Construcción del Dataset
Para el entrenamiento del detector necesitamos tener imágenes del objeto de interés y el etiquetado del mismo. Este etiquetado consiste en las coordenadas de un recuadro que indiquen donde se encuentra los objetos en las imágenes.
![alt text](https://camo.githubusercontent.com/e1e33a7ef92dfc86ab8929dd0e8e96395cbcab5c/68747470733a2f2f686162726173746f726167652e6f72672f66696c65732f3232392f6630362f3237372f32323966303632373766636334393237393334326237656466616262623437612e6a7067)
El formato de las coordenadas normalmente depende de la librería que vamos a usar. Para el caso de Yolo, el etiquetador genera un archivo `.txt` para cada imagen en el mismo directorio con el mismo nombre. El etiquetador se llama [Yolo mark](https://github.com/AlexeyAB/Yolo_mark). Para cada objeto el archivo contiene una nueva línea con el siguiente contenido: 

`<object-class> <x_center> <y_center> <width> <height>`

Donde: 
  * `<object-class>` - número entero que va de `0` to `(classes-1)`
  * `<x_center> <y_center> <width> <height>` - valores flotantes **relativos** al ancho y alto de la imagen, estos van de `(0.0 to 1.0]`
  * por ejemplo: `<x> = <absolute_x> / <image_width>` or `<height> = <absolute_height> / <image_height>`
  * `<x_center> <y_center>` - son los centros del rectángulo (no es una de las esquinas)

Cuando etiquetamos imágenes con Yolo_mark, el segundo parámetro que le pasamos es un archivo en donde se guardarán los nombres de las imágenes etiquetadas. Conservarlo porque lo usaremos más adelante para generar los archivos de entrenamiento y validación.

### Tareas para construir el dataset
 1. Buscar imágenes del objeto de interés.
 1. Etiquetar las imágenes usando Yolo_mark.
 1. Continúa más adelante


## Darknet
Para entrenar un detector de objetos basados en Yolo usaremos el framework [Darknet](https://github.com/pjreddie/darknet).

Después de descargarlo editamos el Makefile y cambiamos las primeras líneas: GPU = 1,CUDNN=1 y OPENCV=1. Con esto activamos el uso de la GPU, del módulo Deep Neural Network de cuda y el uso de opencv.

Además nos bajamos los pesos para Yolov3 preentrenados en ImageNet:
[darknet53.conv.74](https://pjreddie.com/media/files/darknet53.conv.74).
### Configuración de la Red
Crear una copia del archivo **darknet/cfg/yolov3.cfg** en **darknet/cfg/yolov3_custom.cfg**.
Modificar lo siguiente:
```
# línea 8 & 9: 
width = 416, height = 416

# línea 20 
max_batches = 6000 

# línea 22 
steps = 5400 

# línea 603, 689, 776 (convolucional anterior a capa yolo): 
filters = 18

# línea 610, 696, 783 (todas las apariciones): 
classes = 1
```
Para probar el entorno inicial se puede poner **max_batches = 10**. Esto hace que se ejecute 10 iteraciones y guarde el modelo.

En caso de que querramos detectar entre más de un objeto, hay que setear el número de **classes** y setear los filtros con la siguiente fórmula: **filters = (classes + 5)*3**.
### Configuración del dataset
Para entrenar el modelo necesitamos un conjunto de entrenamiento y un conjunto de validación.
Para esto copiamos todas las imágenes en la carpeta **darknet/data/images/**.

Y luego creamos dos archivos, **darknet/data/train.txt** y **darknet/data/val.txt**. Para esto podemos usar el archivo que genera Yolo_mark.

En estos archivos pondremos los paths relativos a las imágenes de entrenamiento y de validación. La cantidad de imágenes de entrenamiento debe ser de aproximadamente el 90%.

Ejemplo de archivo *train.txt*:
```
data/images/000001.png
data/images/000002.png
...
data/images/000090.png
```
Ejemplo de archivo *val.txt*:
```
data/images/000091.png
data/images/000002.png
...
data/images/000100.png
```

Las anotaciones **00XXXX.txt** deben guardarse en un directorio **darknet/data/labels/**.

Crear un archivo **darknet/data/yolo.names** que contenga el nombre del o los objetos a detectar uno en cada línea. El primer nombre de objeto se corresponde con el de `<object-id>` 0 del etiquetado, el segundo con el de `<object-id>` 1, y así sucesivamente.

Por último creamos el archivo **darknet/data/yolo.data** que contendrá lo siguiente:

```
classes= 1 # número de objetos
train  = data/train.txt
valid  = data/val.txt
names = data/yolo.names
backup = backup
```
En backup se guardarán los modelos entrenados. En caso de que hayamos usado otros nombres para los archivos de configuración anteriores, adaptarlos en **yolo.data**.

Por último comprimir la carpeta **darknet** y subirla a drive.

## Repaso de los pasos
 1. Construir dataset.
 1. Descargar darknet.
 1. Modificar el Makefile.
 1. Copiar yolov3.cfg y adaptarla a nuestro problema.
 1. Copiar las imágenes en **darknet/data/images**.
 1. Copiar las etiquetas en **darknet/data/labels**.
 1. Crear los archivos **train.txt** y **val.txt**.
 1. Crear **yolo.names**.
 1. Crear **yolo.data**.
 1. Comprimir **darknet** y subirla a drive.

## Entrenamiento en colab
 * Montamos el drive en colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

 * Chequeamos que esté instalado el compilador de *cuda*

In [ ]:
!nvcc --version

 * Descomprimimos el dataset



In [ ]:
!unzip "/content/drive/My Drive/colab/darknet.zip"

 * Compilamos darknet, lo hacemos ejecutable.

In [ ]:
%cd /content/darknet
!make clean
!make
!chmod +x ./darknet

 * Creamos un enlace simbólico entre colab y nuestro drive para guardar los modelos intermedios. Esto supone que existe una carpeta **colab/backup** en nuestro drive.

In [ ]:
!rm /content/darknet/backup -r
!ln -s /content/drive/'My Drive'/colab/backup /content/darknet


 * Si usamos windows para generar los archivos de configuración, instalamos una utilidad para linuxearlos y se la aplicamos a los archivos. Si usan linux para generar los archivos, háganlo igual porque hace alguna magia.

In [ ]:
!sudo apt install dos2unix

In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom.cfg

Si vemos algún error en la conversión, puede ser porque la pifiamos en la ubicación de los archivos. Revisarlo.


 * Entrenamos nuestro propio modelo. Esto puede tardar aproximadamente un día.

In [ ]:
%cd /content/darknet
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg
#!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg "/content/drive/My Drive/colab/backup/yolov3_custom_train_4000.weights"

## Probamos el modelo
1. Buscar 2 imágenes nuevas.
1. Mostrarlas en pantalla (matplotlib). 
1. Detectar objetos con nuestro modelo.
1. Mostrar el resultado de la detección.

Para este práctico pueden descargar el modelo y probarlo en sus PC.

Completar la siguiente celda con el código para realizar lo anterior.

In [ ]:
# Completar con el código para probar el modelo